## 1. Cleaning and exploratory analysis

Authors : Haddam Yacine, Ka Alioune, Renaud Adrien

<p align="center">
  <a>
    <img src="./figures/logo-hi-paris-retina.png" alt="Logo" width="280" height="180">
  </a>

  <h3 align="center">Data Science Bootcamp</h3>
</p>

Data cleaning
======

#### How can it be problematic for our analyst to use the dataset as is, without cleaning? 

#### WHAT IS DATA CLEANING:
The purpose of this step is to normalize the data to facilitate its manipulation during the analysis.
Several operations are possible: modify or delete data that are incorrect, incomplete, irrelevant, corrupted, duplicated or badly formatted


### Why is this important? 
- Correct duplicate or misfiled data. 
- Correct errors in manual data entry. 
- Wrong data can affect the results and their accuracy.


Objective of this lab
======


Clean the datasets in order to obtain a quality dataset, without errors, duplicates, irrelevant values... ready to be analyzed

### Data Path

`data_dir` is the path to data folder.

In [ ]:
data_dir = "/home/jovyan/personal_workspace/bootcamp/data"

### Libraries

In [ ]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


sys.path.append('../src/notebooks')
from utils.get_data import load_data

pd.set_option('display.max_columns', 500)
warnings.simplefilter(action='ignore', category=FutureWarning) 
warnings.simplefilter(action='ignore', category=UserWarning)

## 1. Building Metadata 

The first file contains a description of the buildings :

* `building_id`: unique identifier for the building.
* `site_id`: unique identifier for the site (multiple buildings are located at the same site).
* `primary_use`: Primary space usage of all buildings is mapped using the [energystar scheme building description types](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/identify-your-property-type). 
* `	sub_primary_use`: [energystar scheme building description types](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/identify-your-property-type) subcategory.
* `square_feet`:Floor area of building in square meters (m2).
* `lat`: Latitude of building location to site level.
* `lng`: Longitude of building location to site level.
* `year_built`: Year corresponding to when building was first constructed, in the format YYYY.
* `floor_count`: Number of floors corresponding to building.

### Read the file

In [ ]:
building_meta = pd.read_feather(
    os.path.join(data_dir, 'raw/building_metadata.feather')
)
building_meta.head()

### How many rows and columns contains the building_metadata file ?

In [ ]:
numbers_rows = building_meta.shape[0]
numbers_columns = building_meta.shape[1]
print(f'Building metadata rows : {numbers_rows} | columns : {numbers_columns}')

### What is the percentage of empty value in each column ? 

In [ ]:
# na values
print('The perc. of empty values')
for column_name in building_meta:
    percentage_na = building_meta[column_name].isna().mean() * 100
    print(f'{column_name} : {np.round(percentage_na)} %')

We drop the two column 'year_build' and 'floor_count'

In [ ]:
# drop columns with high percentage of na values
building_meta = building_meta.drop(columns=['year_built', 'floor_count'])
building_meta.head()

#### 14% of building_id have an NaN (Not a Numerical) values -> drop this rows

In [ ]:
# identify the rows with NaN Lat and Lng
# | : or logique
mask_lat_lng_na = (
    building_meta.lat.isna().values
    | building_meta.lng.isna().values
)

# ~: tilde sign identify a not logique
# filter rows with mmask_lat_lng_na
building_meta = building_meta[~mask_lat_lng_na].reset_index(drop=True)

numbers_rows = building_meta.shape[0]
numbers_columns = building_meta.shape[1]
print(f'Building metadata without na rows rows: {numbers_rows} | columns : {numbers_columns}')

## Is there a duplicates rows ?

In [ ]:
mask_duplicated = building_meta.duplicated(keep=False)
building_meta[mask_duplicated]

We drop all duplicates rows

In [ ]:
building_meta = building_meta.drop_duplicates().reset_index(drop=True)

numbers_rows = building_meta.shape[0]
numbers_columns = building_meta.shape[1]
print(f'Building metadata without duplicates rows : {numbers_rows} | columns : {numbers_columns}')

### How many unique building and site are presented ?

In [ ]:
number_unique_building = building_meta.building_id.nunique()
number_unique_site = building_meta.site_id.nunique()
print(f'Number of unique building {number_unique_building}')
print(f'Number of unique site {number_unique_site}')

### What are the percentage of building in each category of primary_use ?

In [ ]:
perc_bulding_per_primary_use = (
    building_meta
    .groupby('primary_use')
    ['building_id']
    .agg("nunique") 
    / number_unique_building
    * 100
)
perc_bulding_per_primary_use.sort_values(ascending=False)

We notice categories with low percentages.

In [ ]:
building_meta.primary_use.unique()

We grouped them into homogenous categories to facilite the data analysis :
- 'Warehouse/storage', 'Manufacturing/industrial','Technology/science', 'Utility' --> 'Industry'
- 'Religious worship' --> 'Other'
- 'Retail', 'Food sales and service' --> Services

In [ ]:
maping_primary_use_grouped = {
    'Education': 'Education',
    'Lodging/residential': 'Lodging/residential',
    'Office': 'Office',
    'Entertainment/public assembly': 'Entertainment/public assembly',
    'Other': 'Other',
    'Retail': 'Services',
    'Parking': 'Parking',
    'Public services': 'Public services',
    'Warehouse/storage': 'Industry',
    'Food sales and service': 'Services',
    'Religious worship': 'Other',
    'Healthcare': 'Healthcare',
    'Utility': 'Industry',
    'Technology/science': 'Industry',
    'Manufacturing/industrial': 'Industry',
    'Services': 'Services'
}

In [ ]:
building_meta.primary_use = (
    building_meta
    .primary_use
    .map(maping_primary_use_grouped)
)
perc_bulding_per_primary_use = (
    building_meta
    .groupby('primary_use')
    ['building_id']
    .agg("nunique")
    / number_unique_building
    * 100
)
perc_bulding_per_primary_use.sort_values(ascending=False)

### How to verify the quality of geographic data ?

#### Easy ! A map plot 

In [ ]:
site_position = (
    building_meta
    [['site_id', 'lat', 'lng']]
    .drop_duplicates()
    .reset_index(drop=True)
)

fig = px.scatter_mapbox(
    site_position, lat="lat", lon="lng", hover_name="site_id",
    color_discrete_sequence=["fuchsia"], zoom=2, height=300
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

#### We can see an false coordinate fo the building_id 16 !

#### exclude all rows with the building_id == 16

In [ ]:
building_meta = building_meta[building_meta.site_id != 16].reset_index(drop=True)
building_meta.shape

### What about square_feet?

#### Calcul statistics to check continuous variable 

In [ ]:
# get statictis aof the variable
building_meta.square_feet.describe()

#### Plot a distribution 

In [ ]:
_ = sns.displot(building_meta, x='square_feet', kde=True)

### We don't notice any negative values or incoherent distribution 

### Save the result of cleaning building_metadata in the folder clean to use for the next steps

In [ ]:
building_meta.to_feather(
    os.path.join(data_dir, 'clean/building_metadata.feather')
)

## 2. Weather

The second file contains weather data in 2016 and 2017 (every hour) for all the `site_id` referenced in `building_metadata`.

* <code>timestamp</code>: date and time in the format YYYY-MM-DD hh:mm:ss. Local timezone.
* <code>site_id</code>: unique identifier for the site.
* <code>air_temperature</code>: The temperature of the air in degrees Celsius (ºC).
* <code>cloud_coverage</code>: Portion of the sky covered in clouds, in [oktas](https://en.wikipedia.org/wiki/Okta).
* <code>dew_temperature</code>: The dew point (the temperature to which a given parcel of air must be cooled at constant pressure and water vapor content in order for saturation to occur) in degrees Celsius (ºC).
* <code>precip_depth_1_hr</code>: The depth of liquid precipitation that is measured over a one hour accumulation period (mm).
* <code>sea_lvl_pressure</code>: The air pressure relative to Mean Sea Level (MSL) (mbar or hPa).
* <code>wind_direction</code>: The angle, measured in a clockwise direction, between true north and the direction from which the wind is blowing (degrees).
* <code>wind_speed</code>: The rate of horizontal travel of air past a fixed point (m/s).

In [ ]:
weather = pd.read_feather(
    os.path.join(data_dir, 'raw/weather.feather')
)
weather.head()

### Get general information 

In [ ]:
weather.info()

#### The DataFrame is composed of chronological weather measurement (continuous variables). So, we recommend plotting the distribution of each column to have visual sanity check.

In [ ]:
sns.set(rc={'figure.figsize': (24, 12)})
sns.set(font_scale=1.5)
f, axes = plt.subplots(2, 3)
axes = axes.flatten()

color = "dodgerblue"

# "airTemperature" histogram
ax1 = axes[0]
g1 = sns.distplot(weather["air_temperature"].dropna(), ax=ax1, color=color)
ax1.set_title('Air temperature (ºC)')
ax1.set(xlabel="")

# "dewTemperature" histogram
ax2 = axes[1]
g2 = sns.distplot(weather["dew_temperature"].dropna(), ax=ax2, color=color)
ax2.set_title('Dew temperature (ºC)')
ax2.set(xlabel="")

# "precipDepth1HR" histogram
ax3 = axes[2]
g3 = sns.distplot(weather["precip_depth_1_hr"].dropna(),
                  ax=ax3, color=color, kde_kws={'bw': 0.1})
ax3.set_title('Precipitation Depth in 1 hour (mm)')
ax3.set(xlabel="")

# "seaLvlPressure" histogram
ax5 = axes[3]
g5 = sns.distplot(weather["sea_level_pressure"].dropna(), ax=ax5, color=color)
ax5.set_title('Pressure (hPa)')
ax5.set(xlabel="")

# "windSpeed" histogram
ax6 = axes[4]
g6 = sns.distplot(weather["wind_speed"].dropna(), ax=ax6, color=color)
ax6.set_title('Wind speed (m/s)')
ax6.set(xlabel="")

# "windDirection" polar histogram
degrees = weather["wind_direction"]
radians = np.deg2rad(weather["wind_direction"])
bin_size = 20
a, b = np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])
ax7 = f.add_subplot(248, projection='polar')
ax7.set_theta_zero_location("N")
ax7.set_theta_direction("clockwise")
g7 = plt.bar(centers, a, width=np.deg2rad(bin_size),
             bottom=0.0, color=color, alpha=0.6, edgecolor='k')
ax7.set_title('Wind direction', pad=5, loc="left")

# Remove empty axes
f.delaxes(axes[5])

plt.tight_layout()
plt.show()

#### We can see abnormal values of air temperature superior to 50 degrees °C

In [ ]:
# we notice that the high air temperature are in site_id 14
sites_with_temperature_above_50 = (
    weather
    [weather.air_temperature > 50]
    .site_id
    .unique()
    .tolist()
)
print(f'Sites with temperature above 50°C: {sites_with_temperature_above_50}')

After we read the documentation from API data provider, we understood that the air temperature for the site_id 14 is provide in Fahrenheit (°F)

To convert Fahrenheit (°F) to Celsius (°C)  (Fahrenheit - 32) * 5/9

In [ ]:
def f_to_c(fahrenheit):
    return (fahrenheit - 32) * 5/9

We use lambda notation and apply fuction from pandas to correct the air temperature values of the side_id number 14 

In [ ]:
air_temperature_C = (
    weather
    [weather.site_id == 14]
    .air_temperature
    .apply(lambda x: f_to_c(x))
)
weather.loc[weather.site_id == 14, 'air_temperature'] = air_temperature_C

#### Generally, there are missing lines in the chronological data due to an irregular timestamp. Let's check it ! 

In [ ]:
def check_timestamp_contunity(df):
    diff_timestamp = (df.timestamp - df.timestamp.shift(1)) / timedelta(hours=1)
    flag_continuty_timestamp = 'OK' if diff_timestamp.var() == 0 else 'KO'
    return flag_continuty_timestamp

In [ ]:
print('Check timestamp contunity')
for site in weather.site_id.unique():
    df_site = weather[weather.site_id == site].sort_values('timestamp')
    flag_continuty_timestamp = check_timestamp_contunity(df_site)
    print(f'site_id {site}: {flag_continuty_timestamp}')

In [ ]:
site_dfs = []
for site in weather.site_id.unique():
    new_idx = pd.date_range(start='2016-1-1', end='2017-12-31-23', freq='H')
    site_df = weather[weather.site_id == site].set_index('timestamp').reindex(new_idx)
    site_df.site_id = site

    for col in [c for c in site_df.columns if c != 'site_id']:
        site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
        site_df[col] = site_df[col].fillna(weather[col].median())
    site_dfs.append(site_df)

df = pd.concat(site_dfs)
weather = df.reset_index().rename(columns={'index':'timestamp'})

### Save the result of cleaning weather in the folder clean to use for the next steps

In [ ]:
weather.to_feather(
    os.path.join(data_dir, 'clean/weather.feather'),
)

# It's your turn !

## 3. Meters

The third file contains contains the actual energy consumption of the buildings in 2016 and 2017 (every hour).   
There are four different energy types (electricity, chilledwater, steam, hotwater).  
In each line the `meter` variable indicates the type of energy and the `meter_reading` the consumption.



#### Meter readings

- `timestamp`: date and time in the format YYYY-MM-DD hh:mm:ss. 2016 and 2017 data.
- `building_id`: unique identifier for the buildings.
- `meter_reading`: meter reading in kilowatt hour (kWh) .
- `meter`: meter type, `chilledwater`, `electricity`, `hotwater`, `steam`


The `meter` variable is encoded as an integer as follow:
```json
{0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}
```

### TO DO 

1. Read the meters reading file and show the 5 first rows
2. Show general information about meters dataframe
3. Create a loop over name columns to check the existence of nan values
4. Show duplicates rows if exist and drop it
5. Check if there are any inconsistent data in the `meter` column
6. Is there  an uncommon values in meter_reading ?
7. Save the cleaned dataframe

### Hints

- **Hint 1**:

```python
meters = pd.read_feather(
    os.path.join(data_dir, 'raw/meters.feather')
)
meters.head()
```


- **Hint 2**:

```python
meters.info()
````

- **Hint 3**:

```python
print('The perc. of empty values')
for column_name in meters.columns:
    percentage_na = meters[column_name].isna().mean() * 100
    print(f'{column_name} : {np.round(percentage_na)} %')
```

- **Hint 4**:

```python
mask_duplicated = meters.duplicated(keep=False)
meters[mask_duplicated]
meters = meters.drop_duplicates().reset_index(drop=True)

numbers_rows = meters.shape[0]
numbers_columns = meters.shape[1]
print(f'meters without duplicates rows : {numbers_rows} | columns : {numbers_columns}')
```

- **Hint 5**:

```python
# show all unique values of `meter`
print(meters.meter.unique())
# drop the lines with `meter == -1`
meters = meters[meters.meter != -1].reset_index(drop=True)
```

- **Hint 6**:

```python
meters.meter_reading.describe()
meters = meters[meters.meter_reading > 0].reset_index(drop=True)
```

- **Hint 7**:

```python
meters.to_feather(
    os.path.join(data_dir, 'clean/meters.feather')
)
```

### Read the `meters` file

In [ ]:
# TODO 1

### Get general information about the dataset ? (# of rows, # of columns, NaN values)

In [ ]:
# TODO 2

In [ ]:
# TODO 3

### Is there a duplicates rows ?

In [ ]:
# TODO 4

### How many categories of unique meters exist in the dataset ?

Is there an incoherent data ? If yes, drop the concerning rows

In [ ]:
# TODO 5

### What about meter_reading?

Compute statistics to check continuous variable.   
Is there an incoherent data ? If yes, drop the concerning rows

In [ ]:
# TODO 6

### Save the result of cleaning meters in the folder clean to use for the next steps

In [ ]:
# TODO 7

# Take away

- Edit variable types / formats
- Identify duplicates
- Delete columns with many missing values
- Use common sense and keep only relevant variables
- Observe the distribution of values of a variable
- Visual representations are useful to understand how a variable works

### Pitfalls to avoid
- Automatically delete a duplicate: understand why the duplicate appeared
- Automatically delete all rows with missing values and lose information. Approximating some values allows you to keep information to meet an objective.
- Automatically delete outliers: understand where they come from, are they errors or do they only represent extreme cases?
- Retain variables that could be harmful to the ethics of a project (skin color, address...)

### Go Further :
- [The Ultimate Guide to Data Cleaning](https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4)
- [Learn Data Cleaning Tutorials | Kaggle](https://www.kaggle.com/learn/data-cleaning)